Mount to Google Drive

In [1]:
from google.colab import drive
import datetime

# Mount Google Drive
drive.mount('/content/drive')

# Specify the path where you want to save the model
model_save_path = '/content/drive/My Drive/Models/2025-02-27 22:35:50.240524'

Mounted at /content/drive


In [2]:
%%capture
!pip install -U transformers vllm unsloth torch

In [3]:
!nvidia-smi

Sat Mar  1 05:12:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             44W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

Load the model from the drive mount

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from unsloth import FastLanguageModel
import torch
max_seq_length = 512 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

load_path = "/content/drive/My Drive/Models/2025-02-27 22:35:50.240524"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)
model.load_adapter(load_path)
if isinstance(model.config.torch_dtype, str):
    model.config.torch_dtype = getattr(torch, model.config.torch_dtype)

# model = FastLanguageModel.from_pretrained(model, load_path)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/meta-llama-3.1-8b-instruct-unsloth-bnb-4bit with actual GPU utilization = 23.1%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 39.56 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 2.96 GB. Also swap space = 5 GB.
INFO 03-01 05:32:07 config.py:549] This model supports multiple tasks: {'reward', 'classify', 'generate', 'score', 'embed'}. Defaulting to 'generate'.
Unsloth: vLLM Bitsandbytes config usin

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 03-01 05:32:15 model_runner.py:1115] Loading model weights took 5.5664 GB
INFO 03-01 05:32:17 worker.py:267] Memory profiling takes 0.75 seconds
INFO 03-01 05:32:17 worker.py:267] the current vLLM instance can use total_gpu_memory (39.56GiB) x gpu_memory_utilization (0.23) = 9.14GiB
INFO 03-01 05:32:17 worker.py:267] model weights take 5.57GiB; non_torch_memory takes 0.00GiB; PyTorch activation peak memory takes 0.73GiB; the rest of the memory reserved for KV Cache is 2.84GiB.
INFO 03-01 05:32:17 executor_base.py:111] # cuda blocks: 1453, # CPU blocks: 2560
INFO 03-01 05:32:17 executor_base.py:116] Maximum concurrency for 512 tokens per request: 45.41x
INFO 03-01 05:32:17 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utiliz

Capturing CUDA graph shapes: 100%|██████████| 23/23 [00:30<00:00,  1.32s/it]

INFO 03-01 05:32:48 model_runner.py:1562] Graph capturing finished in 30 secs, took 0.18 GiB
INFO 03-01 05:32:48 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 32.31 seconds


Initialize PyPokerEngine rig

In [9]:
!pip install PyPokerEngine

In [6]:
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT },
    {"role" : "user", "content" :
"""
You are a specialist in playing 6-handed No Limit Texas Holdem. The following will be a game scenario and you need to make the optimal decision.

Here is a game summary:

The small blind is 0.5 chips and the big blind is 1 chips. Everyone started with 100 chips.
The player positions involved in this game are UTG, HJ, CO, BTN, SB, BB.
In this hand, your position is HJ, and your holding is [King of Diamond and Jack of Spade].
Before the flop, HJ raise 2.0 chips, and BB call. Assume that all other players that is not mentioned folded.
The flop comes King Of Spade, Seven Of Heart, and Two Of Diamond, then BB check, and HJ check.
The turn comes Jack Of Club, then BB check, HJ bet 3 chips, BB raise 10 chips, and HJ call.
The river comes Seven Of Club, then BB check.


Now it is your turn to make a move.
To remind you, the current pot size is 24.0 chips, and your holding is [King of Diamond and Jack of Spade].

Decide on an action based on the strength of your hand on this board, your position, and actions before you. Do not explain your answer.
Your optimal action is:
"""
     },
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora_weights("grpo_saved_lora"),
)[0].outputs[0].text

output

AttributeError: 'LlamaForCausalLM' object has no attribute 'load_lora_weights'

In [ ]:
model_save_path = "wesleyyliu/pokerzero"
model = AutoModelForCausalLM.from_pretrained(model_save_path)
tokenizer = AutoTokenizer.from_pretrained(model_save_path)

text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
import random
import numpy as np
from pypokerengine.api.game import setup_config, start_poker
from pypokerengine.players import BasePokerPlayer
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer (assumes your model is fine-tuned for poker decisions)
model_save_path = "wesleyyliu/pokerzero"
model = AutoModelForCausalLM.from_pretrained(model_save_path)
tokenizer = AutoTokenizer.from_pretrained(model_save_path)

class TransformerPokerPlayer(BasePokerPlayer):
    def __init__(self):
        super().__init__()
        self.model = model
        self.tokenizer = tokenizer

    def preprocess_state(self, hole_card, round_state):
        """
        Convert the game state into a string prompt for the transformer model.
        This is a simplistic example; you should format it in a way that your model understands.
        """
        board_cards = round_state.get("community_card", [])
        prompt = f"Hole cards: {hole_card}; Board cards: {board_cards}; Action?"
        return prompt

    def map_text_to_action(self, generated_text, valid_actions):
        """
        Map the generated text from the model to a valid action.
        This example assumes that your model outputs a word like 'fold', 'call', or 'raise'.
        """
        # Clean and lower-case the generated text
        action_candidate = generated_text.strip().lower()
        # Check if the predicted action is valid
        for action in valid_actions:
            if action_candidate in action["action"]:
                amount = action.get("amount", 0)
                return action["action"], amount
        # Fallback to random if the prediction doesn't match a valid action.
        fallback = random.choice(valid_actions)
        return fallback["action"], fallback.get("amount", 0)

    def declare_action(self, valid_actions, hole_card, round_state):
        # Preprocess game state to create a text prompt
        prompt = self.preprocess_state(hole_card, round_state)
        inputs = self.tokenizer.encode(prompt, return_tensors='pt')

        # Generate output from the model; adjust parameters as needed
        if not hasattr(model, "max_seq_length"): model.max_seq_length = model.config.max_position_embeddings
        outputs = self.model.generate(inputs, max_new_tokens=20, num_return_sequences=1)
        generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Map the model's output to a valid poker action
        action, amount = self.map_text_to_action(generated_text, valid_actions)
        return action, amount

    def receive_game_start_message(self, game_info):
        print("Game started:", game_info)

    def receive_round_start_message(self, round_count, hole_card, seats):
        print(f"Round {round_count} started with hole cards: {hole_card}")

    def receive_street_start_message(self, street, round_state):
        print(f"Street {street} started.")

    def receive_game_update_message(self, action, round_state):
        pass  # You could store experiences here if needed

    def receive_round_result_message(self, winners, hand_info, round_state):
        print("Round ended. Winners:", winners)

def main():
    # Setup the poker game
    config = setup_config(max_round=50, initial_stack=1000, small_blind_amount=10)
    # Register the transformer-based player
    config.register_player(name="TransformerPlayer1", algorithm=TransformerPokerPlayer())
    config.register_player(name="TransformerPlayer2", algorithm=TransformerPokerPlayer())
    # Start the game simulation
    game_result = start_poker(config, verbose=1)
    print("Final game result:", game_result)

if __name__ == '__main__':
    main()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Game started: {'player_num': 2, 'rule': {'initial_stack': 1000, 'max_round': 50, 'small_blind_amount': 10, 'ante': 0, 'blind_structure': {}}, 'seats': [{'name': 'TransformerPlayer1', 'uuid': 'dtlfansmocuwwvcynrxref', 'stack': 1000, 'state': 'participating'}, {'name': 'TransformerPlayer2', 'uuid': 'ifxskjmubjrnbvnimxglee', 'stack': 1000, 'state': 'participating'}]}
Game started: {'player_num': 2, 'rule': {'initial_stack': 1000, 'max_round': 50, 'small_blind_amount': 10, 'ante': 0, 'blind_structure': {}}, 'seats': [{'name': 'TransformerPlayer1', 'uuid': 'dtlfansmocuwwvcynrxref', 'stack': 1000, 'state': 'participating'}, {'name': 'TransformerPlayer2', 'uuid': 'ifxskjmubjrnbvnimxglee', 'stack': 1000, 'state': 'participating'}]}
Round 1 started with hole cards: ['CT', 'DA']
Round 1 started with hole cards: ['CJ', 'D2']
Street preflop started.
Street preflop started.
Started the round 1
Street "preflop" started. (community card = [])


RuntimeError: Invalid device string: 'float32'